In [2]:
import numpy as np
import pandas as pd
import json

import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import accuracy_score, cohen_kappa_score, f1_score, classification_report

In [4]:
class Net(nn.Module):
    def __init__(self, INPUT_SIZE):
        super(Net, self).__init__()
        # Input is output of resnet18 image classification model
        self.fc1 = nn.Linear(INPUT_SIZE, 500) 
        self.fc11 = nn.Linear(500, 100)
        self.fc2 = nn.Linear(100, 1)
        self.act_out = nn.Sigmoid()

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc11(x))
        x = self.act_out(self.fc2(x))
        return x


def load_dataset_df(label_path):
    label = pd.read_csv(label_path)
    label = label.drop(columns=['Unnamed: 0'])
    dataset = label

    # load embeddings
    with open('./dataset_work/embeddings.json') as f:
        embeddings = json.load(f)
    all_embeddings = pd.DataFrame.from_dict(embeddings, orient='index', columns=['embedding'])
    all_embeddings.index.name = 'id'
    dataset = pd.merge(dataset, all_embeddings, on='id', how='left')

    # load personalities
    with open('./dataset_work/personalities.json') as f:
        personalities = json.load(f)
    all_personalities = pd.DataFrame.from_dict(personalities, orient='index', columns=['playfulness', 'chase-proneness', 'curiosity', 'sociability', 'aggressiveness', 'shyness'])
    all_personalities.index.name = 'id'
    dataset = pd.merge(dataset, all_personalities, on='id', how='left')

    return dataset

def vectorize_dataset(df, with_personalities, with_embeddings):
    if with_personalities and with_embeddings:
        cols = df[['embedding','playfulness', 'chase-proneness', 'curiosity', 'sociability', 'aggressiveness', 'shyness']]
        x = cols.values
        embeddings = x[:,0]
        expanded = np.array(embeddings.tolist())
        x = np.concatenate((expanded, x[:,1:]), axis=1)
        x = np.array(x, dtype='float64')
        y = df['label'].values
    elif with_embeddings:
        cols = df[['embedding','playfulness', 'chase-proneness', 'curiosity', 'sociability', 'aggressiveness', 'shyness']]
        x = cols.values
        embeddings = x[:,0]
        expanded = np.array(embeddings.tolist())
        x = expanded
        y = df['label'].values
    else:
        cols = df[['embedding','playfulness', 'chase-proneness', 'curiosity', 'sociability', 'aggressiveness', 'shyness']]
        x = cols.values
        x = x[:,1:]
        x = np.array(x, dtype="float64")
        y = df['label'].values
    return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32).reshape(-1, 1)

def load(label_path, with_personalities, with_embeddings):
    df = load_dataset_df(label_path)
    return vectorize_dataset(df, with_personalities, with_embeddings)

def pred(label_path):
    PATH = "trained_models/alice_medium_images_True_personalities_False_optm_adam_loss_MSE_EPOCHS_150_BATCH_1000_REG_0.001_LR_0.0001_f1_0.7230682829943155"
    train_x_t, train_y_t = load(label_path, with_personalities=False, with_embeddings=True)
    model = Net(train_x_t.shape[1])
    model.load_state_dict(torch.load(PATH))
    model.eval()
    return model(train_x_t)

In [5]:
pred('dataset_work/labels/personality_only/alice_train_personalityTrue_imageFalse_labels.csv')

tensor([[9.5293e-01],
        [9.5922e-01],
        [9.4612e-01],
        [2.9549e-02],
        [3.7544e-03],
        [1.3778e-03],
        [1.8958e-02],
        [9.9204e-01],
        [9.8160e-01],
        [2.1203e-02],
        [9.9024e-01],
        [9.5098e-01],
        [4.7164e-03],
        [8.7307e-03],
        [8.0175e-02],
        [9.3970e-01],
        [4.9654e-02],
        [3.2673e-03],
        [9.9830e-01],
        [9.3030e-01],
        [2.2449e-04],
        [5.7973e-02],
        [9.6703e-05],
        [3.3359e-02],
        [9.9404e-01],
        [9.4235e-01],
        [9.8731e-01],
        [8.9633e-02],
        [9.6525e-01],
        [4.2601e-02],
        [9.5643e-01],
        [1.7876e-02],
        [5.9314e-02],
        [6.5777e-03],
        [9.4922e-01],
        [9.2829e-01],
        [2.2593e-02],
        [3.1117e-02],
        [9.7913e-01],
        [2.3658e-02],
        [9.5574e-01],
        [6.2806e-04],
        [1.6636e-02],
        [3.6489e-02],
        [4.0206e-02],
        [2